In [1]:
# from calx.calx import Calx # clean up later
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics

## Fetching data, training a classifier

This is an example of using the calx library, following the steps outline in the "Lime - basic usage, two class case" notebook. This is to ensure some kind of feature parity

As per the tutorial, will be using a 2-class subset (atheism and christianity) from the [20 newsgroups dataset](http://scikit-learn.org/stable/datasets/#the-20-newsgroups-text-dataset).

In [2]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'soc.religion.christian']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)
class_names = ['atheism', 'christian']

Use the tfidf vectorizer

In [3]:
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False)
train_vectors = vectorizer.fit_transform(newsgroups_train.data)
test_vectors = vectorizer.transform(newsgroups_test.data)

Use random forest classifier

In [4]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
rf.fit(train_vectors, newsgroups_train.target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [5]:
pred = rf.predict(test_vectors)
sklearn.metrics.f1_score(newsgroups_test.target, pred, average='binary')

0.92200232828870776

The classifier achieves a high F score as expected, but this could indicate that the model overfits the training data.

## Explaining predictions

In [6]:
from sklearn.pipeline import make_pipeline
c = make_pipeline(vectorizer, rf)

In [7]:
print(c.predict_proba([newsgroups_test.data[0]]))

[[ 0.336  0.664]]


In [10]:
from calx.calx import Calx # clean up later
explainer = Calx(class_names)

In [11]:
idx = 83
exp = explainer.explain_instance(newsgroups_test.data[idx], c.predict_proba, num_features=6)
print('Document id: %d' % idx)
print('Probability(christian) =', c.predict_proba([newsgroups_test.data[idx]])[0,1])
print('True class: %s' % class_names[newsgroups_test.target[idx]])

TypeError: no supported conversion for types: (dtype('<U454'),)